<font face='georgia'>
    
   <h4><strong>What does tf-idf mean?</strong></h4>

   <p>    
Tf-idf stands for <em>term frequency-inverse document frequency</em>, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. Variations of the tf-idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query.
</p>
    
   <p>
One of the simplest ranking functions is computed by summing the tf-idf for each query term; many more sophisticated ranking functions are variants of this simple model.
</p>
    
   <p>
Tf-idf can be successfully used for stop-words filtering in various subject fields including text summarization and classification.
</p>
    
</font>

<font face='georgia'>
    <h4><strong>How to Compute:</strong></h4>

Typically, the tf-idf weight is composed by two terms: the first computes the normalized Term Frequency (TF), aka. the number of times a word appears in a document, divided by the total number of words in that document; the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

 <ul>
    <li>
<strong>TF:</strong> Term Frequency, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization: <br>

$TF(t) = \frac{\text{Number of times term t appears in a document}}{\text{Total number of terms in the document}}.$
</li>
<li>
<strong>IDF:</strong> Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following: <br>

$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}}.$
for numerical stabiltiy we will be changing this formula little bit
$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}+1}.$
</li>
</ul>

<br>
<h4><strong>Example</strong></h4>
<p>

Consider a document containing 100 words wherein the word cat appears 3 times. The term frequency (i.e., tf) for cat is then (3 / 100) = 0.03. Now, assume we have 10 million documents and the word cat appears in one thousand of these. Then, the inverse document frequency (i.e., idf) is calculated as log(10,000,000 / 1,000) = 4. Thus, the Tf-idf weight is the product of these quantities: 0.03 * 4 = 0.12.
</p>
</font>

<font face='georgia'>
    <h4><strong>1. Building a TFIDF Vectorizer & comparing its results with Sklearn:</strong></h4>

<ul>
    <li> As a part of this mini task we will be implementing TFIDF vectorizer on a collection of text documents.</li>
    <br>
    <li> We will compare the results of our own implementation of TFIDF vectorizer with that of sklearns implemenation TFIDF vectorizer.</li>
    <br>
    <li> Sklearn does few more tweaks in the implementation of its version of TFIDF vectorizer, so to replicate the exact results we will need to add following things into our custom implementation of tfidf vectorizer:
       <ol>
        <li> Sklearn has its vocabulary generated from idf sorted in alphabetical order</li>
        <li> Sklearn's formula of idf is different from the standard textbook formula. Here the constant <strong>"1"</strong> is added to the numerator and denominator of the idf as if an extra document was seen containing every term in the collection exactly once, which prevents zero divisions.
            
 $IDF(t) = 1+\log_{e}\frac{1\text{ }+\text{ Total  number of documents in collection}} {1+\text{Number of documents with term t in it}}.$
        </li>
        <li> Sklearn applies L2-normalization on its output matrix.</li>
        <li> The final output of sklearn tfidf vectorizer is a sparse matrix.</li>
    </ol>


### Corpus

In [ ]:
## SkLearn# Collection of string documents

corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

### SkLearn Implementation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
skl_output = vectorizer.transform(corpus)

In [ ]:
# sklearn feature names, they are sorted in alphabetic order by default.

print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [ ]:
# Here we will print the sklearn tfidf vectorizer idf values after applying the fit method
# After using the fit function on the corpus the vocab has 9 words in it, and each has its idf value.

print(vectorizer.idf_)

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


In [ ]:
# shape of sklearn tfidf vectorizer output after applying transform method.

skl_output.shape

(4, 9)

In [ ]:
# sklearn tfidf values for first line of the above corpus.
# Here the output is a sparse matrix

print(skl_output[0])

  (0, 8)	0.38408524091481483
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045


In [ ]:
# sklearn tfidf values for first line of the above corpus.
# To understand the output better, here we are converting the sparse output matrix to dense matrix and printing it.
# Notice that this output is normalized using L2 normalization. sklearn does this by default.

print(skl_output[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


### Our custom implementation

In [ ]:
# Import required libraries
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy as np



def fit(dataset):
    """
    This function returns the dictionary of all the unique words in the corpus and the idf values for each
    of the unique words.
    *******************************************************************************************************
    Input: "dataset": A list of sentences
    Output: returns vocab, idf_values
    where: vocab - is a dictionary containing all the unique words as keys and the column index as values
           idf_values - also a dictionary consisting of all the unique words and their idf values as key:value pairs
    """
    
    if isinstance(dataset, (list,)):
        unique_words = set()
        for row in dataset:
            for word in row.split():
                if len(word)<2:
                    continue
                unique_words.add(word)
        unique_words = sorted(list(unique_words))
        vocab = {j:i for i,j in enumerate(unique_words)}
        
        # This is the implementation of idf
        N = len(dataset)                      # Total no.of documents

        # We need to find the idf values for each word in the vocab 
        # We need the count the number of documents with a particular word(of the vocab)
        idf_freq = {x:0 for x in vocab.keys()}
        for term in vocab.keys():              # for each word in the vocab
            for row in dataset:                # for each document in the corpus
                if term in row.split():
                    idf_freq[term]+=1          # count if the word is present in the document
        
        # Calculating the idf value
        idf_ = {x:0 for x in vocab.keys()}
        for term in idf_.keys():
            idf_[term] = 1+ np.log((1+N)/(1+idf_freq[term]))      # Sklearn's modified formula for caclculating idf values
        
        return vocab,idf_
    else:
        print("Please pass the input as a list of sentences")
        

# implementing the tf_idf
# for this we need to have the dataset
def transform(dataset):
    """
    This function returns the tf_idf vectors for each document in the corpus
    ********************************************************************************
    Input: The corpus or 'dataset' which should be a list of sentences.
    Output: This function returns the tf_idf vector for each document in the corpus.
    
    """
    rows = []
    columns = []
    values = []
    if isinstance(dataset,(list,)):
        vocab, idf_vals = fit(dataset)             # Fetching the unique words and the IDF values.

        for idx,row in enumerate(dataset):
            # the problem is that we need to use each column of the vocabulary 

            tf = dict(Counter(row.split()))         # this is counting the frequency of the term in a particular row

            s = sum(tf.values())                    # this is to calculate the total number of words in a row 

            tf = {x:tf[x]/s for x in tf.keys()}     # Dividing the values by the total number of words in a row
       

            tf_idf = {x:0 for x in vocab.keys()}    # Creating an empty
            for i in vocab.keys():
                # We are using try except block to assign tf_idf values for each document in the corpus. Why?
                # The reason being that in a document it is not necessary that all the unique words will be present,
                # some of them will not be, which will not be counted while calculating the tf values. 
                # So if a word in not present in a document then it won't be present in the term freq dictionary, and will
                # throw a KerError to us when we try to acces that word in the dictionary. 
                # We are usign this fact to assign the tf_idf(word) = 0(when word in not present in a document)
                try:
                    tf_idf[i] = tf[i]*idf_vals[i]   # calculating the tf_idf values
                except KeyError:
                    tf_idf[i] = 0

# For a particular row we are going to store the tf_idf values for words that are present in that particular row/document only
# So it is logical to iterate over the words that are present in the term frequency dictionary(since it is specific to a row)
# discard all the other words present in the vocabulary.
            
            for word in tf.keys():                         # for each word present in a particular document
                if len(word)<2:
                    continue
                # replace a with vocab
                col_idx = vocab.get(word, -1)              # get the column index for the word
                if col_idx!=-1:
                    # Store the row,column index and the tf_idf values
                    columns.append(col_idx)                
                    rows.append(idx)
                    values.append(tf_idf[word])
        
        values = csr_matrix((values, (rows,columns)), shape=(len(corpus), len(vocab)))
        values = normalize(values)                         # Normalising the values in the sparse matrix
        return values
    else:
        print('Pass the input as a list of sentences')



In [ ]:
vocab, idf_vals= fit(corpus)
out = transform(corpus)

# Compairing the results with Sklearn's implementation

In [ ]:
# The unique words in a corpus
print("Custom Implementation:")
print(list(vocab.keys()))
print("Sklearn's Implementation:")
print(vectorizer.get_feature_names())

Custom Implementation:
['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
Sklearn's Implementation:
['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [ ]:
# Idf values
print("Custom Implementation:")
print(list(idf_vals.values()))
print("Sklearn's Implementation:")
print(list(vectorizer.idf_))

Custom Implementation:
[1.916290731874155, 1.2231435513142097, 1.5108256237659907, 1.0, 1.916290731874155, 1.916290731874155, 1.0, 1.916290731874155, 1.0]
Sklearn's Implementation:
[1.916290731874155, 1.2231435513142097, 1.5108256237659907, 1.0, 1.916290731874155, 1.916290731874155, 1.0, 1.916290731874155, 1.0]


In [ ]:
# TF-IDF values for the first document
print("Custom Implementation:")
print(out[0].toarray())
print("Sklearn's Implementation:")
print(skl_output[0].toarray())

Custom Implementation:
[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]
Sklearn's Implementation:
[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


<font face='georgia'>
    <h4><strong>2. Implement max features functionality:</strong></h4>

<ul>
    <li> As a part of this task we will modify our fit and transform functions so that our vocab will contain only 50 terms with top idf scores.</li>
    <br>
    <li>This task is similar to our previous task, just that here our vocabulary is limited to only top 50 features names based on their idf values. Basically our output will have exactly 50 columns and the number of rows will depend on the number of documents we have in our corpus.</li>
    <br>
    <li>Here we have a pickle file, with file name <strong>cleaned_strings</strong>. We have to load the corpus from this file and use it as input to our tfidf vectorizer.</li>

In [ ]:
# Below is the code to load the cleaned_strings pickle file provided
# Here corpus is of list type

import pickle
with open('cleaned_strings', 'rb') as f:
    corpus = pickle.load(f)
    
# printing the length of the corpus loaded
print("Number of documents in corpus = ",len(corpus))

Number of documents in corpus =  746


In [ ]:
# Write your code here.
# Try not to hardcode any values.
# Make sure its well documented and readble with appropriate comments.


def fit(dataset):
    """
    This function only returns the vocabulary/dictionary of all the unique words in the corpus
    *******************************************************************************************************
    Input: "dataset": A list of sentences
    Output: returns vocab, idf_values
    where: vocab - is a dictionary containing all the unique words as keys and the column index as values
           idf_values - also a dictionary consisting of all the unique words and their idf values as key:value pairs
    """
    
    if isinstance(dataset, (list,)):
        # These lines of code are just to find the set of unique words and give them a column number in a sorted order. 
        unique_words = set()
        for row in dataset:
            for word in row.split():
                if len(word)<2:
                    continue
                unique_words.add(word)
        unique_words = sorted(list(unique_words))
        vocab = {j:i for i,j in enumerate(unique_words)}
        
        # The implementation of idf starts here
        N = len(dataset)                      # Total no.of documents

        # We need to find the idf values for each word in the vocab 
        # We need the count the number of documents with a particular word(of the vocab)
        idf_freq = {x:0 for x in vocab.keys()}
        for term in vocab.keys():
            for row in dataset:
                if term in row.split():
                    idf_freq[term]+=1
        
        # Calculating the idf value
        idf_ = {x:0 for x in vocab.keys()}
        for term in idf_.keys():                                      # for each unique word in the corpus 
            idf_[term] = 1+ np.log((1+N)/(1+idf_freq[term]))          # calculating the idf values
        
        n = 50    # the reduced size of the vocabulary
        f = [(idf_[x],x) for x in idf_.keys()]                  # storing the idf,words as a list of tuples
        f = sorted(f,reverse=True)[0:n]                # sorting by the idf values and taking only the top 50 idf value words
        new_idf = {x:y for y,x in f}
        new_vocab = {i:j for j,i in enumerate(new_idf.keys())}   # Storing the words and the idf values in a dictionary
        
        return new_vocab,new_idf
    else:
        print("Please pass the input as a list of sentences")
        

# implementing the tf_idf
# for this we need to have the dataset
def transform(dataset):
    """
    This function returns the tf_idf vectors for each document in the corpus
    ********************************************************************************
    Input: The corpus or 'dataset' which should be a list of sentences.
    Output: This function returns the tf_idf vector for each document in the corpus.
    
    """
    # Implementing the tf_idf freq
    
    rows = []
    columns = []
    values = []
    if isinstance(dataset,(list,)):
        
        vocab, idf_vals = fit(dataset)                      # get the dictionary of unique words and their column index
                                                            # get the idf values for each unique word in the corpus
        
        for idx,row in enumerate(dataset):

            tf = dict(Counter(row.split()))         # this is counting the frequency of the words in a particular document

            s = sum(tf.values())                    # this is to calculate the total number of words in a document

            tf = {x:tf[x]/s for x in tf.keys()}     # Normalising the sum 
       

            tf_idf = {x:0 for x in vocab.keys()}    # Creating an empty dictionary to store the tf_idf values
            for i in vocab.keys():
                # We are using try except block to assign tf_idf values for each document in the corpus. Why?
                # The reason being that in a document it is not necessary that all the unique words will be present,
                # some of them will not be, which will not be counted while calculating the tf values. 
                # So if a word in not present in a document then it won't be present in the term freq dictionary, and will
                # throw a KerError to us when we try to acces that word in the dictionary. 
                # We are usign this fact to assign the tf_idf(word) = 0(when word in not present in a document)
                try:
                    tf_idf[i] = tf[i]*idf_vals[i]   # calculating the tf_idf values
                except KeyError:
                    tf_idf[i] = 0

# For a particular row we are going to store the tf_idf values for words that are present in that particular row/document only
# So it is logical to iterate over the words that are present in the term frequency dictionary(since it is specific to a row)
# discard all the other words present in the vocabulary cause their tf_idf values will be zero anyway.
            
            for word in tf.keys():                         # for each word present in a particular document
                if len(word)<2:
                    continue
                # replace a with vocab
                col_idx = vocab.get(word, -1)              # get the column index for the word
                if col_idx!=-1:
                    # Store the row,column index and the tf_idf values
                    columns.append(col_idx)                
                    rows.append(idx)
                    values.append(tf_idf[word])
        
        values = csr_matrix((values, (rows,columns)), shape=(len(corpus), len(vocab)))
        values = normalize(values)                         # Normalising the values in the sparse matrix
        return values
    else:
        print('Pass the input as a list of sentences')


In [ ]:
vocab, idf = fit(corpus)
tf_idf = transform(corpus)

In [ ]:
print("The words with top idf values in descending order:")
print(list(vocab.keys()))

The words with top idf values in descending order:
['zombiez', 'zillion', 'yun', 'youtube', 'youthful', 'younger', 'yelps', 'yawn', 'yardley', 'wrote', 'writers', 'wrap', 'wow', 'woven', 'wouldnt', 'worthy', 'worthwhile', 'worthless', 'worry', 'worked', 'woo', 'wont', 'wong', 'wondered', 'woa', 'witticisms', 'within', 'wise', 'win', 'wily', 'willie', 'william', 'wild', 'wih', 'wife', 'widmark', 'wide', 'whoever', 'whites', 'whine', 'whenever', 'went', 'welsh', 'weight', 'wedding', 'website', 'weaving', 'weariness', 'weaker', 'wb']


In [ ]:
print("The idf values in descending order:")
print(list(idf.values()))

The idf values in descending order:
[6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872, 6.922918004572872]


In [ ]:
# Output for a single document
print(tf_idf[135])

  (0, 11)	0.37796447300922725
  (0, 24)	0.37796447300922725
  (0, 25)	0.37796447300922725
  (0, 27)	0.37796447300922725
  (0, 33)	0.37796447300922725
  (0, 45)	0.37796447300922725
  (0, 47)	0.37796447300922725


In [ ]:
# Converting the output to a dense representation
print(tf_idf[135].toarray())

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.37796447
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.37796447 0.37796447 0.         0.37796447 0.         0.
  0.         0.         0.         0.37796447 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.37796447 0.         0.37796447
  0.         0.        ]]
